In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, decomposition
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from catboost import *
import catboost
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from random import choice
from scipy.stats import randint as sp_randint
import gc
from sklearn.model_selection import TimeSeriesSplit, KFold
pd.options.display.max_columns = None
%matplotlib inline

In [9]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))
data = pd.read_csv('data/full_data.csv')
features = ['id', 'atm_group', 'city_type', 'city_rank', 'city_type_rank', 'dbcluster',
            'geo_city', 'region', 'street', 'dbcluster', 'KMeans','MiniBatchKMeans',
            'DBSCAN', 'Birch', 'MeanShift']
for f in features:
    data[f] = data[f].values.astype('str')
    
test = pd.read_csv('data/test.csv')

#data['city_type'] = data['city_type'].values.astype('str')
#data['city_type_rank'] = data['city_type_rank'].values.astype('str')
#data['city_rank'] = data['city_rank'].values.astype('str')
#data['atm_group'] = data['atm_group'].values.astype('str')
#data['dbcluster'] = data['dbcluster'].values.astype('str')
#data['id'] = data['id'].values.astype('str')

In [10]:
X = data[data.isTrain == True].drop(['target', 'isTrain', 'address', 'address_rus', 'city'], 1)
Y = data[data.isTrain == True][['target']]
X_test = data[data.isTrain == False].drop(['target', 'isTrain', 'address', 'address_rus', 'city'], 1)

In [11]:
cat_features = []
for i in features:
    cf = X.columns.get_loc(i)
    cat_features.append(cf)

Random Grid Search CV

In [26]:
n_iterations = 10 # number of iterations for random search
top_n = 9 # select top n parameter sets

rmse_mean = []
rmse_std = []
dict_list = []
# prepare indexes for stratified cross validation
skf = KFold(n_splits=5, shuffle=False)
skf.get_n_splits(X, Y)

5

In [ ]:
# loop for random search
print ("Random search start...")
print ("")

for i in range(0, n_iterations):
    skf_split = skf.split(X, Y)
    param_dist = {'depth': choice([3, 6, 8, 10, 16]),
                  'learning_rate': choice([0.3, 0.1, 0.05, 0.01]),
                  'l2_leaf_reg': choice([1, 3, 5, 7, 10]),
                  'bagging_temperature': choice([0,1,5,7,10]),
                  #'bootstrap_type': choice(['Bayesian', 'Bernoulli']),
                  #'subsample': choice([0, 0.66, 1]),
                  'random_strength': choice([1, 3, 5, 7]),
                  'rsm': choice([0.1,0.3,0.5,0.7,1]),
                  #'fold_len_multiplier': choice([1, 2, 5, 7]),
                  'loss_function': 'RMSE',
                  'iterations': 1000} 
    
    rmse_metric = []
    
    print ("Cycle {}...".format(i+1))
    for train_index, test_index in skf_split:
    
        X_train = X.iloc[train_index]
        y_train = Y.iloc[train_index]
    
        X_val = X.iloc[test_index]
        y_val = Y.iloc[test_index]
    
        # training
        model = CatBoostRegressor(**param_dist)
        model.fit(X_train, y_train, cat_features=cat_features,
                 eval_set=[(X_train, y_train), (X_val, y_val)], 
                 early_stopping_rounds=100, verbose=1000)
        # predicting
        y_pred = model.predict(X_val, ntree_end=model.best_iteration_)
        
        rmse_i = rmse(y_val, y_pred)
        rmse_metric.append(rmse_i)
   
    print ("Finish cycle {}...".format(i+1))

    rmse_array = np.asarray(rmse_metric)
    
    rmse_mean.append(rmse_array.mean())
    rmse_std.append(rmse_array.std())
    dict_list.append(param_dist)
    gc.collect()
    print('New iteration')

results_pd = pd.DataFrame({"rmse_mean": rmse_mean,
                           "rmse_std": rmse_std,
                           "parameters": dict_list})

Random search start...

Cycle 1...
0:	learn: 0.0825366	test: 0.0824228	test1: 0.0858336	best: 0.0858336 (0)	total: 14.4ms	remaining: 14.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04776715447
bestIteration = 838

Shrink model to first 839 iterations.
0:	learn: 0.0834669	test: 0.0834444	test1: 0.0827698	best: 0.0827698 (0)	total: 15.3ms	remaining: 15.3s
999:	learn: 0.0449948	test: 0.0442589	test1: 0.0447859	best: 0.0447846 (996)	total: 11.4s	remaining: 0us

bestTest = 0.04478455335
bestIteration = 996

Shrink model to first 997 iterations.
0:	learn: 0.0839672	test: 0.0839606	test1: 0.0831249	best: 0.0831249 (0)	total: 14.9ms	remaining: 14.9s
999:	learn: 0.0450968	test: 0.0438075	test1: 0.0453703	best: 0.0453685 (996)	total: 13.2s	remaining: 0us

bestTest = 0.0453685497
bestIteration = 996

Shrink model to first 997 iterations.
0:	learn: 0.0838718	test: 0.0838660	test1: 0.0851953	best: 0.0851953 (0)	total: 20.2ms	remaining: 20.2s
999:	learn: 0.0448905	test: 0.

In [ ]:
z = results_pd.sort_values('rmse_mean').copy().reset_index()
print("Minimum mean RMSE : ", z.rmse_mean.min(), "\nAnd the parameters list is : \n", z.parameters[0])

Train catboost

In [12]:
params = {'depth': 3, 'iterations': 100000, 
        'learning_rate': 0.1, 'random_seed': 1111,
        'loss_function': 'RMSE'}
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=False, random_state=42)
# Cleaning and defining parameters for LGBM
model = CatBoostRegressor(**params)

In [13]:
prediction = np.zeros(test.shape[0])

for fold_n, (train_index, test_index) in enumerate(folds.split(X)):
    print('Fold:', fold_n)
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_valid = Y.iloc[train_index], Y.iloc[test_index]
    #train_pool = Pool(data=X_train, label=Y_train, cat_features=cat_features)
    #val_pool = Pool(data=X_valid, label=Y_valid, cat_features=cat_features)

    model.fit(X_train, Y_train, cat_features=cat_features,
            eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
            early_stopping_rounds=100, verbose=500)
    
    y_pred = model.predict(X_test, ntree_end=model.best_iteration_)
    prediction += y_pred
prediction /= n_fold

Fold: 0
0:	learn: 0.0819241	test: 0.0819131	test1: 0.0653144	best: 0.0653144 (0)	total: 76.3ms	remaining: 2h 7m 10s
500:	learn: 0.0415648	test: 0.0389923	test1: 0.0495886	best: 0.0495864 (498)	total: 5.2s	remaining: 17m 13s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04926541063
bestIteration = 831

Shrink model to first 832 iterations.
Fold: 1
0:	learn: 0.0781122	test: 0.0780916	test1: 0.0974687	best: 0.0974687 (0)	total: 13.9ms	remaining: 23m 12s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04377114071
bestIteration = 87

Shrink model to first 88 iterations.
Fold: 2
0:	learn: 0.0821966	test: 0.0821881	test1: 0.0600900	best: 0.0600900 (0)	total: 14.3ms	remaining: 23m 46s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04571370511
bestIteration = 247

Shrink model to first 248 iterations.
Fold: 3
0:	learn: 0.0783726	test: 0.0783519	test1: 0.0951434	best: 0.0951434 (0)	total: 13.5ms	remaining: 22m 34s
Stopped by overf

In [21]:
submission = pd.DataFrame(prediction, index=test.index, columns=['target'])

In [22]:
submission.reset_index(level=0, inplace=True)
submission.index.name = ""

In [25]:
submission.to_csv('catboost_submit_18.12.2018.csv',index=False)

In [24]:
submission.head()

,index,target
,,
0,0,-0.038721
1,1,-0.043709
2,2,-0.016416
3,3,-0.043427
4,4,-0.000173
